ARMA (Auto Regression Moving Average)  

In [ ]:
import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sb 
%matplotlib inline 

from statsmodels.tsa.ar_model import AutoReg, ar_select_order  
from sklearn.metrics import mean_squared_error 

In [ ]:
# min_tmp = pd.read_csv('../Datasets/daily-min-temperatures.csv', header=0, parse_dates=[0], index_col=0) 
# min_tmp = pd.read_csv('../Datasets/daily-min-temperatures.csv', header=0, parse_dates=[0])  

# min_tmp = pd.read_csv('../Datasets/daily-min-temperatures.csv', header=0, parse_dates=['Date'], date_format=dateparse)   

dateparse = lambda x: pd.to_datetime(x, format='%Y-%m-%d') 
min_tmp = pd.read_csv('../Datasets/daily-min-temperatures.csv', header=0, parse_dates=[0], date_format=dateparse)   
min_tmp 

In [ ]:
if not isinstance(min_tmp.index, pd.DatetimeIndex):
    min_tmp.index = pd.to_datetime(min_tmp.index)
    print('Converted index to DateTime format.') 

# min_tmp = min_tmp.asfreq('D') 
# min_tmp = min_tmp.asfreq('H')  
# min_tmp 

# Moving Average -  

Let, actual value (at time t+1) = y  
     predicted value (at time t+1) = y_ + residuals  

y = y_ + residuals  (residual = forecasting error = y - y_)  
Residuals should be white noise with no patterns and seasonality.  

Our model may leave info in residuals too. So to extract the info, we run a forecating model on residuals. The 2nd level model for residuals will identify and capture in the new forecast and this new forecast will be added to our original forecast for better forecast.  

use of AutoReg with moving avg. is also called ARMA method.  

In [ ]:
min_tmp['t'] = min_tmp['Temp'].shift(1)        # this column is the lagged column 
min_tmp 

In [ ]:
min_tmp['Resid'] = min_tmp['Temp'] - min_tmp['t'] 
min_tmp 

Test and Train split - 

In [ ]:
# Series data on the Residual values 

train, test = min_tmp.Resid[1: (min_tmp.shape[0]-7)], min_tmp.Resid[(min_tmp.shape[0]-7):]  

In [ ]:
train 

In [ ]:
test 

In [ ]:
train_y, test_y = min_tmp.Temp[1: (min_tmp.shape[0]-7)], min_tmp.Temp[(min_tmp.shape[0]-7):]  
train_x, test_x = min_tmp.Date[1: (min_tmp.shape[0]-7)], min_tmp.Date[(min_tmp.shape[0]-7):]  

print(f" y_train size : {train_y.shape} \n y_test size : {test_y.shape}") 
print(f" y_train : {train_y} \n y_test : {test_y}") 

print(f" x_train size : {train_x.shape} \n x_test size : {test_x.shape}") 
print(f" x_train : {train_x} \n x_test : {test_x}") 

Auto-Regression -  

In [ ]:
# axlag is less than the number of observations in training data 
# max_lag = max(10, 29, 30, len(train) - 1)  
max_lag = min(10, 29, 30, len(train) - 1)  

# Automatically selecting Optimal number of lagged values 

slct_ord = ar_select_order(train, maxlag=max_lag, ic='aic', seasonal=False) 
optimal_lags = slct_ord.ar_lags 
print(f"optimal number of lags : {optimal_lags}")  

max_optimal_lag = max(optimal_lags) if optimal_lags else 0 
print(f"Max OPtimal lags : {max_optimal_lag}") 

In [ ]:
# train = train[max_optimal_lag:]  
model = AutoReg(train, lags=max_optimal_lag)     
model_fit = model.fit() 
model_fit.summary()  

In [ ]:
model_fit.params 

Predictions -  

In [ ]:

pred_resid = model_fit.predict(start=len(train), end=(len(train) + len(test) - 1)) 
pred_resid 
# Predicting forecast residuals for the next values in the time series 

In [ ]:
pred = min_tmp.t[(min_tmp.shape[0]-7):] + pred_resid 
pred 
# Forecast naive values 

In [ ]:
mse_test = mean_squared_error(test_y, pred) 
print(f"Mean Squared Error (Test) : {mse_test}") 

In [ ]:
test.index.strftime('%Y-%m-%d') 

In [ ]:

plt.plot(test_x, test_y, label="Actual") 
plt.plot(test_x, pred, color='red', linestyle='dashed', label="Predicted")  

plt.legend() 
plt.show() 